# **Importing Required Libraries**

In [83]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# **Step 1 : Data Loading**

In [84]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
df = pd.read_csv(url, header=None, names=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
           'occupation', 'relationship', 'race', 'sex', 'capital-gain',
           'capital-loss', 'hours-per-week', 'native-country', 'income']
, na_values=' ?', skipinitialspace=True, delimiter=',')

In [85]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# **Step 2 : Initial Data Inspection and Cleaning**

### Step 2.1 : Display dataset information and preview data

**Preview Data:** Using head() lets students see the actual records, which is critical for understanding the context of the data.

**Dataset Info:** The info() function shows non-null counts and datatypes, which helps in quickly spotting missing values or incorrect data formats.



In [86]:
# Display dataset information and preview data
# -> Display the first 5 rows and print the basic states for data
# -> Hint: head and info
df.info()
# print()
# df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


**Dataset Dimensions:** Knowing the shape of the data informs students about its scale, which can affect computation time and choice of algorithms.

In [87]:
# Display the dimension of the data
# -> Hint : shape
df.shape

(32561, 15)

### **Step 2.2 : Basic Statistical Summary:**

The describe() function provides vital statistics (min, max, mean, standard deviation) that help identify any outliers or anomalies in numerical columns.

In [88]:
# Check the min , max , count , means , standard diviation etc
# Hint -> describe
df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


# **Step 2.3 : Counting Missing Values:**

This step is essential for diagnosing data quality. Missing values can lead to biased or inaccurate models if not handled properly.

In [89]:
# Count missing values in each column
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

### **Step 2.4 : Checking for Duplicate Records:**

Duplicates can skew the analysis by over-representing some data, so it's important to remove them.

In [90]:
# Check for duplicate records
# print(df.duplicated())
print("Duplicated Value Count : ",df.duplicated().sum())
# Drop duplicate rows if found
df = df.drop_duplicates()

Duplicated Value Count :  24


### **Step 2.5 : Inspecting Unique Values in Categorical Columns:**

Unique value inspection reveals if there are any unexpected values (e.g., a '?' or extra spaces) that need cleaning. This is important for ensuring reliable encoding later.

In [91]:
# Check the unique data of each Categorical column to find if there is any irrelevant record or data e.g one record contains ? mark

# Check unique values in each categorical column
for column in df.select_dtypes(include=['object']).columns:
    print(f'{column}: {df[column].unique()}')

workclass: ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked']
education: ['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']
marital-status: ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed']
occupation: ['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' '?'
 'Protective-serv' 'Armed-Forces' 'Priv-house-serv']
relationship: ['Not-in-family' 'Husband' 'Wife' 'Own-child' 'Unmarried' 'Other-relative']
race: ['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other']
sex: ['Male' 'Female']
native-country: ['United-States' 'Cuba' 'Jamaica' 'India' '?' 'Mexico' 'South'
 'Puerto-Rico' 'Honduras' 'England' '

In [92]:
for column in df.select_dtypes(include=['object']).columns:
    mode = df[column].mode()[0]  # Calculate mode for the current column
    df[column] = df[column].replace('?', mode)  # Replace '?' with mode
    print(f'{column}: {df[column].unique()}')  # Print unique values to verify

workclass: ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov'
 'Self-emp-inc' 'Without-pay' 'Never-worked']
education: ['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']
marital-status: ['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed']
occupation: ['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' 'Protective-serv'
 'Armed-Forces' 'Priv-house-serv']
relationship: ['Not-in-family' 'Husband' 'Wife' 'Own-child' 'Unmarried' 'Other-relative']
race: ['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other']
sex: ['Male' 'Female']
native-country: ['United-States' 'Cuba' 'Jamaica' 'India' 'Mexico' 'South' 'Puerto-Rico'
 'Honduras' 'England' 'Canada' 'Ger

### **Step 2.6 : Validating and Converting Data Types:**

Converting columns to the correct datatype (like converting an ID column to a string) prevents errors in operations such as merging, filtering, or encoding.

In [93]:
# Check the data type of each column and if wrong datatype convert it to the suitable datatype
df.dtypes


age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

***None of the data types need conversion***

### **Step 2.7 : Checking Value Counts for Categorical Columns:**

Value counts help in understanding the distribution within each category. They are useful for detecting class imbalances and anomalies.

In [94]:
# Check value count for each Cateorical column
for column in df.select_dtypes(include=['object']).columns:
    print(f'{column}:\n{df[column].value_counts()}\n')

workclass:
workclass
Private             24509
Self-emp-not-inc     2540
Local-gov            2093
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: count, dtype: int64

education:
education
HS-grad         10494
Some-college     7282
Bachelors        5353
Masters          1722
Assoc-voc        1382
11th             1175
Assoc-acdm       1067
10th              933
7th-8th           645
Prof-school       576
9th               514
12th              433
Doctorate         413
5th-6th           332
1st-4th           166
Preschool          50
Name: count, dtype: int64

marital-status:
marital-status
Married-civ-spouse       14970
Never-married            10667
Divorced                  4441
Separated                 1025
Widowed                    993
Married-spouse-absent      418
Married-AF-spouse           23
Name: count, dtype: int64

occupation:
occupation
Prof-specialty       5979
Craft-repair        

### **Step 2.8 : Handling Missing Values using SimpleImputer:**

Imputation preserves the dataset size while ensuring that no null values interfere with analysis. Different strategies are used for numerical (mean) and categorical (mode) columns based on their characteristics.

In [95]:
# TODO: fill null values either by mean , median  or mode based on type of data

# Separate numeric and categorical columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Impute missing values
imputer_numeric = SimpleImputer(strategy='mean')

# Numeric: fill with mean

# Categorical: fill with most frequent (mode)

In [96]:
# # TODO: fill null values either by mean , median  or mode based on type of data
# # Create a copy of the DataFrame to avoid modifying the original
# df_copy = df.copy()
# print("Before filling missing values:\n", df_copy.isnull().sum())


# # Separate numeric and categorical columns
# numeric_cols = df_copy.select_dtypes(include=[np.number]).columns
# categorical_cols = df_copy.select_dtypes(include=['category']).columns

# # Numeric: fill with mean
# numeric_imputer = SimpleImputer(missing_values=np.nan,strategy="mean")
# df_copy[numeric_cols] = numeric_imputer.fit_transform(df_copy[numeric_cols])

# # Categorical: fill with most frequent (mode)
# categorical_imputer = SimpleImputer(missing_values="?", strategy='most_frequent')
# df_copy[categorical_cols] = categorical_imputer.fit_transform(df_copy[categorical_cols])

# print("After filling missing values:\n", df_copy.isnull().sum())

**Already Replaced ? in categorical cols with mode**

# **Step 3 : Converting Data Types and Cleaning Categorical Data**

### **Step 3.1 :Removing Leading/Trailing Spaces:**

Standardize entries in categorical columns so that no extra spaces lead to misclassification of similar values.



In [97]:
# Check for leading/trailing spaces in categorical data and remove them
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].str.strip()

### **Step 3.2 : Checking and Converting Data Types:**

Ensure that every column is of the correct data type (e.g., IDs as strings, dates as datetime).

In [98]:
# Check the data type of each column and if wrong datatype convert it to the suitable datatype
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

**Already Converted to Suitable types!**

### **3.3 : Converting to 'category' Datatype:**

Transform columns that represent categorical data (like Gender or Embarked port) into the 'category' type to optimize memory and computational performance.

In [99]:
# Convert suitable columns to 'category' datatype
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].astype('category')
df.dtypes

age                  int64
workclass         category
fnlwgt               int64
education         category
education-num        int64
marital-status    category
occupation        category
relationship      category
race              category
sex               category
capital-gain         int64
capital-loss         int64
hours-per-week       int64
native-country    category
income            category
dtype: object

# **Step 4 : Feature Engineering**

### **Step 4.1 : Creating "age_group" Feature:**

Binning converts continuous age values into meaningful categories (e.g., 'Young', 'Adult') that are easier to analyze and interpret.

In [100]:
# create feature "age_group" age categories using binning
bins = [0, 25, 45, 65, np.inf]
labels = ['Young', 'Adult', 'Middle-Aged', 'Senior']

df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)


### **Step 4.2 : Creating "education_hours_interaction" Feature:**

Interaction features help capture complex relationships between variables. In this case, the interaction between education (via 'education-num') and work intensity ('hours-per-week') may reveal underlying patterns related to social or economic outcomes.

In [101]:
# Create an interaction feature "education_hours_interaction": education-num multiplied by hours-per-week (as a proxy for workload vs. education level)
df['education_hours_interaction'] = df['education-num'] * df['hours-per-week']

# **Step 5 : Encoding Categorical Data**

**One-Hot Encoding:** Converts multiple categorical values into binary columns to prevent ordinality.


In [102]:
# One-hot encode the categorical columns (sex, workclass, education, etc.).

from sklearn.preprocessing import OneHotEncoder
import pandas as pd


# Select categorical columns
categorical_cols = df.select_dtypes(include=['category']).columns
#Create OneHotEncoder instance with sparse=False and handle_unknown='ignore'
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# Fit and transform the data
encoded_data = encoder.fit_transform(df[categorical_cols])
#Create column names for encoded features
encoded_cols = encoder.get_feature_names_out(input_features=categorical_cols)
#Create a DataFrame from the encoded data with original index
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=df.index)
#Concatenate encoded and original DataFrames
final_df = pd.concat([df, encoded_df], axis=1)
#Drop the original categorical columns
final_df = final_df.drop(columns=categorical_cols)
# Display the first few rows of the final DataFrame
final_df.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,education_hours_interaction,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income_<=50K,income_>50K,age_group_Adult,age_group_Middle-Aged,age_group_Senior,age_group_Young
0,39,77516,13,2174,0,40,520,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,50,83311,13,0,0,13,169,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,38,215646,9,0,0,40,360,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,53,234721,7,0,0,40,280,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,28,338409,13,0,0,40,520,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [103]:
# ct = OneHotEncoder(sparse_output=False)
# x=pd.DataFrame(ct.fit_transform(df.iloc[:,[0,3]]))
# x.index=df.index


**Label Encoding for Income:** Maps income to binary labels for binary classification tasks.

In [104]:
# Use label encoding for the income column, converting ≤50K to 0 and 50K to 1.

# dictionary to map income values to numerical labels
income_mapping = {'<=50K': 0, '>50K': 1}
# Apply the mapping to the 'income' column
df['income'] = df['income'].map(income_mapping)

In [105]:
label_encoder = LabelEncoder()
df['income'] = label_encoder.fit_transform(df['income'])

# **Step 6 : Normalization and Standardization**

Standardization transforms the specified columns to a mean of 0 and a standard deviation of 1, which is important to ensure comparability among numerical features during model training.

In [106]:
# Standardize the "age", "hours-per-week", "capital-gain" and "capital-loss" column to have a mean of 0 and a standard deviation of 1.
df.head()
# Create a StandardScaler object
scaler = StandardScaler()
# Select the columns to standardize
cols_to_standardize = ['age', 'hours-per-week', 'capital-gain', 'capital-loss']
# Fit the scaler to the selected columns and transform them
df[cols_to_standardize] = scaler.fit_transform(df[cols_to_standardize])

df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,age_group,education_hours_interaction
0,0.030390,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,0.148292,-0.216743,-0.035664,United-States,0,Adult,520
1,0.836973,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,-0.145975,-0.216743,-2.222483,United-States,0,Middle-Aged,169
2,-0.042936,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,-0.145975,-0.216743,-0.035664,United-States,0,Adult,360
3,1.056950,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,-0.145975,-0.216743,-0.035664,United-States,0,Middle-Aged,280
4,-0.776193,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,-0.145975,-0.216743,-0.035664,Cuba,0,Adult,520
